In [45]:
import pyspark
from datetime import datetime, timedelta


spark = pyspark.sql.SparkSession.builder.appName("auctions").getOrCreate()
df_auctions = spark.read.csv('auctions.csv', header=True)
df_installs = spark.read.csv('installs.csv', header=True)
rdd_auctions = df_auctions.rdd
rdd_installs = df_installs.rdd


In [46]:
rdd_auctions.take(5)

[Row(date=u'2019-04-23 18:58:00.842116', device_id=u'2564673204772915246', ref_type_id=u'1', source_id=u'0'),
 Row(date=u'2019-04-23 18:58:01.530771', device_id=u'4441121667607578179', ref_type_id=u'7', source_id=u'0'),
 Row(date=u'2019-04-23 18:58:01.767562', device_id=u'7721769811471055264', ref_type_id=u'1', source_id=u'0'),
 Row(date=u'2019-04-23 18:58:02.363468', device_id=u'6416039086842158968', ref_type_id=u'1', source_id=u'0'),
 Row(date=u'2019-04-23 18:58:02.397559', device_id=u'1258642015983312729', ref_type_id=u'1', source_id=u'0')]

In [47]:
rdd_auctions1 = rdd_auctions.map(lambda x: (datetime.strptime(x[0], "%Y-%m-%d %H:%M:%S.%f"), x[1]))

In [48]:
rdd_auctions2 = rdd_auctions1.map(lambda x: (x[0], x[1], x[0].strftime("%H"), x[0].strftime("%M")))

In [49]:
def getRange(row):
    a = None
    for i in range(0,6):
        if i*10<=int(row[3]) and int(row[3])<=(i+1)*10:
            a = i
    return (row[0], row[1], row[2], row[3], a)

In [51]:
rdd_auctions3 = rdd_auctions2.map(getRange)

In [53]:
def toCSVLine(data):
    try:
        return ','.join(str(d) for d in data)
    except:
        return ','.join('' for d in data)

In [54]:
rdd_auctions4 = rdd_auctions3.map(toCSVLine)
rdd_auctions4.repartition(1).saveAsTextFile('auction_with_range.csv')

In [ ]:
# Installs

In [5]:
df_installs = spark.read.csv('installs.csv', header=True)
rdd_installs = df_installs.rdd

In [6]:
rdd_installs1 = rdd_installs.map(lambda x: (datetime.strptime(x[0], "%Y-%m-%d %H:%M:%S.%f"), \
                                  x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11], x[12], x[13],\
                                           x[14], x[15], x[16], x[17]))

In [7]:
rdd_installs.take(1)

[Row(created=u'2019-04-24 06:23:29.495', application_id=u'1', ref_type=u'1494519392962156891', ref_hash=u'4716708407362582887', click_hash=None, attributed=u'False', implicit=u'True', device_countrycode=u'6287817205707153877', device_brand=None, device_model=u'3.739127126472163e+17', session_user_agent=u'adjust.com', user_agent=None, event_uuid=u'79837499-2f2a-4605-a663-e322f759424f', kind=u'app_open', wifi=None, trans_id=None, ip_address=u'4243443387795468703', device_language=u'3.3013777759776993e+18')]

In [8]:
rdd_installs2 = rdd_installs1.map(lambda x: (x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11], x[12], x[13],\
                                           x[14], x[15], x[16], x[17],x[0].strftime("%H"), x[0].strftime("%M")))

In [9]:
def getRange(row):
    a = None
    for i in range(0,6):
        if i*10<=int(row[19]) and int(row[19])<=(i+1)*10:
            a = i
    return (row[0], row[1], row[2], row[3], row[4], row[5],\
            row[6], row[7], row[8], row[9], row[10], row[11], row[12],\
            row[13], row[14], row[15], row[16], row[17], row[18], row[19], a)

In [10]:
rdd_installs3 = rdd_installs2.map(getRange)

In [11]:
rdd_installs3.take(1)

[(datetime.datetime(2019, 4, 24, 6, 23, 29, 495000),
  u'1',
  u'1494519392962156891',
  u'4716708407362582887',
  None,
  u'False',
  u'True',
  u'6287817205707153877',
  None,
  u'3.739127126472163e+17',
  u'adjust.com',
  None,
  u'79837499-2f2a-4605-a663-e322f759424f',
  u'app_open',
  None,
  None,
  u'4243443387795468703',
  u'3.3013777759776993e+18',
  '06',
  '23',
  2)]

In [12]:
def toCSVLine(data):
    try:
        return ','.join(str(d).replace(",", "") for d in data)
    except:
        return ','.join('' for d in data)

In [13]:
rdd_installs4 = rdd_installs3.map(toCSVLine)
rdd_installs4.repartition(1).saveAsTextFile('installs_with_range.csv')

In [14]:
import pandas as pd

In [64]:
installs_range = pd.read_csv('installs_with_range.csv', header=None, names=["created","application_id","ref_type","device_id","click_hash","attributed","implicit","device_countrycode","device_brand","device_model","session_user_agent","user_agent","event_uuid","kind","wifi","trans_id","ip_address","device_language", "hour", "minutes", "range"])


In [59]:
auctions_range = pd.read_csv('auctions_with_range.csv', names=["date", "device_id", "hour", "minute", "range"])

In [65]:
installs_range.head()

,created,application_id,ref_type,device_id,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,...,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language,hour,minutes,range
0,2019-04-24 06:23:29.495000,1.0,1.494519e+18,4.716708e+18,None,False,True,6.287817e+18,None,3.739127126472163e+17,...,None,79837499-2f2a-4605-a663-e322f759424f,app_open,None,None,4.243443e+18,3.3013777759776993e+18,6.0,23.0,2.0
1,2019-04-24 02:06:01.032000,1.0,1.494519e+18,7.143569e+18,None,False,False,6.287817e+18,None,7.80553892759877e+18,...,None,None,None,None,None,4.724289e+18,3.3013777759776993e+18,2.0,6.0,0.0
2,2019-04-20 10:15:36.274000,1.0,1.494519e+18,5.230323e+18,None,False,True,6.287817e+18,None,8.355495513718673e+18,...,None,dda99e3c-9c4b-487d-891c-79f0a02cb4a8,app_open,None,None,8.291809e+18,4.060929664968129e+18,10.0,15.0,1.0
3,2019-04-20 21:56:47.151000,1.0,1.494519e+18,5.097164e+18,None,False,True,6.287817e+18,None,2.3557720913769155e+18,...,None,7010c3ce-0fcf-46c6-9be8-374cc0e20af4,app_open,None,None,4.006812e+18,3.3013777759776993e+18,21.0,56.0,5.0
4,2019-04-20 22:40:41.239000,1.0,1.494519e+18,6.328028e+18,None,False,False,6.287817e+18,None,6.156971151807135e+18,...,None,None,None,None,None,3.386455e+18,3.3013777759776993e+18,22.0,40.0,4.0


In [67]:
auctions_range.head()

,date,device_id,hour,minute,range
0,2019-04-23 18:58:00.842116,2564673204772915246,18,58,5
1,2019-04-23 18:58:01.530771,4441121667607578179,18,58,5
2,2019-04-23 18:58:01.767562,7721769811471055264,18,58,5
3,2019-04-23 18:58:02.363468,6416039086842158968,18,58,5
4,2019-04-23 18:58:02.397559,1258642015983312729,18,58,5


In [68]:
#ME QUEDO solo con los 3 primeros dias para uqe la distancia maxima sea de 3 dias
#arranca el 2019-04-18 00:00:00 asi que el date limite es 2019-04-20 23:59:59
limit_date_train = datetime(2019, 4, 20, 23, 59, 59, 999)
#uso los siguientes 3 dias para testear
limit_date_test_begin = datetime(2019, 4, 21, 0, 0, 0, 0)
limit_date_test_end = datetime(2019, 4, 23, 23, 59, 59, 999)

In [69]:
auctions_range["date"] = pd.to_datetime(auctions_range["date"])
auctions_train = auctions_range[auctions_range["date"] < limit_date_train]
auctions_test = auctions_range[(auctions_range["date"] > limit_date_test_begin) & (auctions_range["date"] < limit_date_test_end)]

In [72]:
#auctions_train.merge(installs_range, how="inner", on=["device_id","hour", "range"])
# 539 conversiones

In [73]:
auctions_train.merge(installs_range, how="left", on=["device_id","hour", "range"])

,date,device_id,hour,minute,range,created,application_id,ref_type,click_hash,attributed,...,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language,minutes
0,2019-04-20 23:57:27.912838,1109595589636746168,23,57,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-04-20 23:57:28.381114,5896614299191635403,23,57,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-04-20 23:57:28.515423,4172466725848941608,23,57,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-04-20 23:57:28.700884,2616279795187318849,23,57,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-04-20 23:57:28.868312,8034952072073026056,23,57,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2019-04-20 23:57:29.202778,8018738201130876812,23,57,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2019-04-20 23:57:29.441246,4914962826064649332,23,57,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2019-04-20 23:57:29.644078,3249069566262483358,23,57,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2019-04-20 23:57:29.896508,3352762403977995643,23,57,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2019-04-20 23:57:30.453524,9003488718010194664,23,57,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
